<a href="https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a align="left" href="https://ultralytics.com/yolov5" target="_blank">
<img width="1024", src="https://user-images.githubusercontent.com/26833433/125273437-35b3fc00-e30d-11eb-9079-46f313325424.png"></a>

This is the **official YOLOv5 🚀 notebook** by **Ultralytics**, and is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/). 
For more information please visit https://github.com/ultralytics/yolov5 and https://ultralytics.com. Thank you!

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-48-g84a8099 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.2/166.8 GB disk)


# 1. Inference

`detect.py` runs YOLOv5 inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases), and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image 
                          vid.mp4  # video
                          path/  # directory
                          path/*.jpg  # glob
                          'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                          'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

detect: weights=['yolov5s.pt'], source=data/images, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-48-g84a8099 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
image 1/2 /content/yolov5/data/images/bus.jpg: 640x480 4 persons, 1 bus, Done. (0.007s)
image 2/2 /content/yolov5/data/images/zidane.jpg: 384x640 2 persons, 1 tie, Done. (0.007s)
Speed: 0.5ms pre-process, 6.9ms inference, 1.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://user-images.githubusercontent.com/26833433/127574988-6a558aa1-d268-44b9-bf6b-62d4c605cc72.jpg" width="600">

# 2. Validate
Validate a model's accuracy on [COCO](https://cocodataset.org/#home) val or test-dev datasets. Models are downloaded automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases). To show results by class use the `--verbose` flag. Note that `pycocotools` metrics may be ~1% better than the equivalent repo metrics, as is visible below, due to slight differences in mAP computation.

## COCO val
Download [COCO val 2017](https://github.com/ultralytics/yolov5/blob/74b34872fdf41941cddcf243951cdb090fbac17b/data/coco.yaml#L14) dataset (1GB - 5000 images), and test model accuracy.

In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

  0%|          | 0.00/780M [00:00<?, ?B/s]

In [ ]:
# Run YOLOv5x on COCO val
!python val.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65 --half

val: data=/content/yolov5/data/coco.yaml, weights=['yolov5x.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-48-g84a8099 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

100% 166M/166M [00:03<00:00, 54.1MB/s]

Fusing layers... 
Model Summary: 444 layers, 86705005 parameters, 0 gradients
val: Scanning '../datasets/coco/val2017' images and labels...4952 found, 48 missing, 0 empty, 0 corrupted: 100% 5000/5000 [00:01<00:00, 2636.64it/s]
val: New cache created: ../datasets/coco/val2017.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 157/157 [01:12<00:00,  2.17it/s]
                 all       5000      36335      0.729       0.63      0.683      0.496
Speed: 0.1ms pre-process, 4.9ms inference, 1.9ms NMS per image a

## COCO test
Download [COCO test2017](https://github.com/ultralytics/yolov5/blob/74b34872fdf41941cddcf243951cdb090fbac17b/data/coco.yaml#L15) dataset (7GB - 40,000 images), to test model accuracy on test-dev set (**20,000 images, no labels**). Results are saved to a `*.json` file which should be **zipped** and submitted to the evaluation server at https://competitions.codalab.org/competitions/20794.

In [ ]:
# Download COCO test-dev2017
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017labels.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip
!f="test2017.zip" && curl http://images.cocodataset.org/zips/$f -o $f && unzip -q $f -d ../datasets/coco/images

In [ ]:
# Run YOLOv5x on COCO test
!python val.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65 --half --task test

# 3. Train

<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="1000" src="https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/615627e5824c9c6195abfda9_computer-vision-cycle.png"/></a></p>
Close the active learning loop by sampling images from your inference conditions with the `roboflow` pip package
<br><br>

Train a YOLOv5s model on the [COCO128](https://www.kaggle.com/ultralytics/coco128) dataset with `--data coco128.yaml`, starting from pretrained `--weights yolov5s.pt`, or from randomly initialized `--weights '' --cfg yolov5s.yaml`.

- **Pretrained [Models](https://github.com/ultralytics/yolov5/tree/master/models)** are downloaded
automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases)
- **[Datasets](https://github.com/ultralytics/yolov5/tree/master/data)** available for autodownload include: [COCO](https://github.com/ultralytics/yolov5/blob/master/data/coco.yaml), [COCO128](https://github.com/ultralytics/yolov5/blob/master/data/coco128.yaml), [VOC](https://github.com/ultralytics/yolov5/blob/master/data/VOC.yaml), [Argoverse](https://github.com/ultralytics/yolov5/blob/master/data/Argoverse.yaml), [VisDrone](https://github.com/ultralytics/yolov5/blob/master/data/VisDrone.yaml), [GlobalWheat](https://github.com/ultralytics/yolov5/blob/master/data/GlobalWheat2020.yaml), [xView](https://github.com/ultralytics/yolov5/blob/master/data/xView.yaml), [Objects365](https://github.com/ultralytics/yolov5/blob/master/data/Objects365.yaml), [SKU-110K](https://github.com/ultralytics/yolov5/blob/master/data/SKU-110K.yaml).
- **Training Results** are saved to `runs/train/` with incrementing run directories, i.e. `runs/train/exp2`, `runs/train/exp3` etc.
<br><br>

## Train on Custom Data with Roboflow 🌟 NEW

[Roboflow](https://roboflow.com/?ref=ultralytics) enables you to easily **organize, label, and prepare** a high quality dataset with your own custom data. Roboflow also makes it easy to establish an active learning pipeline, collaborate with your team on dataset improvement, and integrate directly into your model building workflow with the `roboflow` pip package.

- Custom Training Example: [https://blog.roboflow.com/how-to-train-yolov5-on-a-custom-dataset/](https://blog.roboflow.com/how-to-train-yolov5-on-a-custom-dataset/?ref=ultralytics)
- Custom Training Notebook: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/yolov5-custom-training-tutorial/blob/main/yolov5-custom-training.ipynb)
<br>

<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="480" src="https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/6152a275ad4b4ac20cd2e21a_roboflow-annotate.gif"/></a></p>Label images lightning fast (including with model-assisted labeling)

In [ ]:
# Tensorboard  (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train

In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-48-g84a8099 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, sc

# 4. Visualize

## Weights & Biases Logging 🌟 NEW

[Weights & Biases](https://wandb.ai/site?utm_campaign=repo_yolo_notebook) (W&B) is now integrated with YOLOv5 for real-time visualization and cloud logging of training runs. This allows for better run comparison and introspection, as well improved visibility and collaboration for teams. To enable W&B `pip install wandb`, and then train normally (you will be guided through setup on first use). 

During training you will see live updates at [https://wandb.ai/home](https://wandb.ai/home?utm_campaign=repo_yolo_notebook), and you can create and share detailed [Reports](https://wandb.ai/glenn-jocher/yolov5_tutorial/reports/YOLOv5-COCO128-Tutorial-Results--VmlldzozMDI5OTY) of your results. For more information see the [YOLOv5 Weights & Biases Tutorial](https://github.com/ultralytics/yolov5/issues/1289). 

<p align="left"><img width="900" alt="Weights & Biases dashboard" src="https://user-images.githubusercontent.com/26833433/135390767-c28b050f-8455-4004-adb0-3b730386e2b2.png"></p>

## Local Logging

All results are logged by default to `runs/train`, with a new experiment directory created for each new training as `runs/train/exp2`, `runs/train/exp3`, etc. View train and val jpgs to see mosaics, labels, predictions and augmentation effects. Note an Ultralytics **Mosaic Dataloader** is used for training (shown below), which combines 4 images into 1 mosaic during training.

> <img src="https://user-images.githubusercontent.com/26833433/131255960-b536647f-7c61-4f60-bbc5-cb2544d71b2a.jpg" width="700">  
`train_batch0.jpg` shows train batch 0 mosaics and labels

> <img src="https://user-images.githubusercontent.com/26833433/131256748-603cafc7-55d1-4e58-ab26-83657761aed9.jpg" width="700">  
`test_batch0_labels.jpg` shows val batch 0 labels

> <img src="https://user-images.githubusercontent.com/26833433/131256752-3f25d7a5-7b0f-4bb3-ab78-46343c3800fe.jpg" width="700">  
`test_batch0_pred.jpg` shows val batch 0 _predictions_

Training results are automatically logged to [Tensorboard](https://www.tensorflow.org/tensorboard) and [CSV](https://github.com/ultralytics/yolov5/pull/4148) as `results.csv`, which is plotted as `results.png` (below) after training completes. You can also plot any `results.csv` file manually:

```python
from utils.plots import plot_results 
plot_results('path/to/results.csv')  # plot 'results.csv' as 'results.png'
```

<img align="left" width="800" alt="COCO128 Training Results" src="https://user-images.githubusercontent.com/26833433/126906780-8c5e2990-6116-4de6-b78a-367244a33ccf.png">

# Environments

YOLOv5 may be run in any of the following up-to-date verified environments (with all dependencies including [CUDA](https://developer.nvidia.com/cuda)/[CUDNN](https://developer.nvidia.com/cudnn), [Python](https://www.python.org/) and [PyTorch](https://pytorch.org/) preinstalled):

- **Google Colab and Kaggle** notebooks with free GPU: <a href="https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a> <a href="https://www.kaggle.com/ultralytics/yolov5"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>
- **Google Cloud** Deep Learning VM. See [GCP Quickstart Guide](https://github.com/ultralytics/yolov5/wiki/GCP-Quickstart)
- **Amazon** Deep Learning AMI. See [AWS Quickstart Guide](https://github.com/ultralytics/yolov5/wiki/AWS-Quickstart)
- **Docker Image**. See [Docker Quickstart Guide](https://github.com/ultralytics/yolov5/wiki/Docker-Quickstart) <a href="https://hub.docker.com/r/ultralytics/yolov5"><img src="https://img.shields.io/docker/pulls/ultralytics/yolov5?logo=docker" alt="Docker Pulls"></a>


# Status

![CI CPU testing](https://github.com/ultralytics/yolov5/workflows/CI%20CPU%20testing/badge.svg)

If this badge is green, all [YOLOv5 GitHub Actions](https://github.com/ultralytics/yolov5/actions) Continuous Integration (CI) tests are currently passing. CI tests verify correct operation of YOLOv5 training ([train.py](https://github.com/ultralytics/yolov5/blob/master/train.py)), testing ([val.py](https://github.com/ultralytics/yolov5/blob/master/val.py)), inference ([detect.py](https://github.com/ultralytics/yolov5/blob/master/detect.py)) and export ([export.py](https://github.com/ultralytics/yolov5/blob/master/export.py)) on macOS, Windows, and Ubuntu every 24 hours and on every commit.


# Appendix

Optional extras below. Unit tests validate repo functionality and should be run on any PRs submitted.


In [ ]:
# Reproduce
for x in 'yolov5n', 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x':
  !python val.py --weights {x}.pt --data coco.yaml --img 640 --task speed  # speed
  !python val.py --weights {x}.pt --data coco.yaml --img 640 --conf 0.001 --iou 0.65  # mAP

In [ ]:
# PyTorch Hub
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Images
dir = 'https://ultralytics.com/images/'
imgs = [dir + f for f in ('zidane.jpg', 'bus.jpg')]  # batch of images

# Inference
results = model(imgs)
results.print()  # or .show(), .save()

In [ ]:
# CI Checks
%%shell
export PYTHONPATH="$PWD"  # to run *.py. files in subdirectories
rm -rf runs  # remove runs/
for m in yolov5n; do  # models
  python train.py --img 64 --batch 32 --weights $m.pt --epochs 1 --device 0  # train pretrained
  python train.py --img 64 --batch 32 --weights '' --cfg $m.yaml --epochs 1 --device 0  # train scratch
  for d in 0 cpu; do  # devices
    python val.py --weights $m.pt --device $d # val official
    python val.py --weights runs/train/exp/weights/best.pt --device $d # val custom
    python detect.py --weights $m.pt --device $d  # detect official
    python detect.py --weights runs/train/exp/weights/best.pt --device $d  # detect custom
  done
  python hubconf.py  # hub
  python models/yolo.py --cfg $m.yaml  # build PyTorch model
  python models/tf.py --weights $m.pt  # build TensorFlow model
  python export.py --img 64 --batch 1 --weights $m.pt --include torchscript onnx  # export
done

In [ ]:
# Profile
from utils.torch_utils import profile

m1 = lambda x: x * torch.sigmoid(x)
m2 = torch.nn.SiLU()
results = profile(input=torch.randn(16, 3, 640, 640), ops=[m1, m2], n=100)

In [ ]:
# Evolve
!python train.py --img 640 --batch 64 --epochs 100 --data coco128.yaml --weights yolov5s.pt --cache --noautoanchor --evolve
!d=runs/train/evolve && cp evolve.* $d && zip -r evolve.zip $d && gsutil mv evolve.zip gs://bucket  # upload results (optional)

In [ ]:
# VOC
for b, m in zip([64, 64, 64, 32, 16], ['yolov5n', 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x']):  # batch, model
  !python train.py --batch {b} --weights {m}.pt --data VOC.yaml --epochs 50 --img 512 --hyp hyp.VOC.yaml --project VOC --name {m} --cache

In [ ]:
# TensorRT 
# https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html#installing-pip
!pip install -U nvidia-tensorrt --index-url https://pypi.ngc.nvidia.com  # install
!python export.py --weights yolov5s.pt --include engine --imgsz 640 640 --device 0  # export
!python detect.py --weights yolov5s.engine --imgsz 640 640 --device 0  # inference

In [7]:
import pyrealsense2 as rs
import numpy as np
import cv2
import torch
import pandas as pd
import time
import math

def bincount_app(a):
    a2D = a.reshape(-1,a.shape[-1])
    col_range = (256, 256, 256) # generically : a2D.max(0)+1
    a1D = np.ravel_multi_index(a2D.T, col_range)
    return np.unravel_index(np.bincount(a1D).argmax(), col_range)

def get_filter_gray(depth_colormap,variance):
    #RGB
    try:
        min1=int(depth_colormap[:][:][0].mean())-variance
        max1=int(depth_colormap[:][:][0].mean())+variance
        #print(int(depth_colormap[:][:][0].mean()),int(depth_colormap[:][:][1].mean()),int(depth_colormap[:][:][2].mean()))
        return (max1),(min1)
    except:
        return (18),(1)

def get_filter(depth_colormap,variance):
    #RGB
    Rmax,Gmax,Bmax=255,255,255
    try:
        R=depth_colormap[0].mean()
        G=depth_colormap[1].mean()
        B=depth_colormap[2].mean()
        print(R,G,B)
        theta_1=math.atan(G/B)
        theta_2=math.atan((Gmax-G)/(Bmax-B))
        theta_3=math.asin((Rmax-R)/(variance))
        theta_4=math.asin(R/variance)
        alpha=variance*math.cos(theta_3)
        gamma=variance*math.cos(theta_4)
        Rlow=R-variance*math.cos(theta_3)
        Glow=G-gamma*math.cos(theta_1)
        Blow=B-gamma*math.sin(theta_1)
        Rhigh=R+variance*math.sin(theta_4)
        Ghigh=G+alpha*math.sin(theta_2)
        Bhigh=B+alpha*math.cos(theta_2)
        return (Rhigh,Ghigh,Bhigh),(Rlow,Glow,Blow)
    except:
        return (18, 255, 255),(1, 190, 200)

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom

x1=100
y1=100
light_blue = (1, 190, 200)
dark_blue = (18, 255, 255)
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)
n=0
try:
    while True:
        n+=1
        #print(n)
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        height, width= depth_image.shape
        #print(height, width)
        depth_image = depth_image[y1:height-y1+10, x1:width-130]
        height, width = depth_image.shape
        #print(height, width)
        #break
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.05), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            depth_colormap = cv2.resize(depth_colormap, dsize=(color_colormap_dim[1], color_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            results = model(color_image)
            contours=results.pandas().xyxy[0][results.pandas().xyxy[0]['confidence']>0.6]
            #gray_image=cv2.cvtColor(color_image,cv2.COLOR_BGR2GRAY)
            for x,y,w,h,name,confidence in zip(contours['xmin'].to_numpy().tolist(),contours['ymin'].to_numpy().tolist(),contours['xmax'].to_numpy().tolist(),contours['ymax'].to_numpy().tolist(),contours['name'].to_numpy().tolist(),contours['confidence'].to_numpy().tolist()):
#                 depth_colormap_croped = depth_colormap.copy()
#                 depth_colormap_croped=depth_colormap_croped[int(x):int(w), int(y):int(h)]
#                 color_image_croped=color_image[int(x):int(w), int(y):int(h)]
#                 print(name)
#                 max_color,min_color=get_filter(depth_colormap_croped,10)
#                 try:      
#                     #depth_colormap_croped=cv2.cvtColor(depth_colormap_croped,cv2.COLOR_BGR2GRAY)
#                     hsv_real = cv2.cvtColor(depth_colormap_croped, cv2.COLOR_RGB2HSV)
#                     max_color,min_color=get_filter(depth_colormap_croped,10)
                    
#                     mask = cv2.inRange(hsv_real, min_color, max_color)
#                     image = cv2.bitwise_and(depth_colormap_croped,depth_colormap_croped, mask=mask)
#                     mask = np.where((image==(0,0,0)).all(axis=2), 0, 255).astype(np.uint8)
#                     # put mask into alpha cha
#                     result = image.copy()
#                     result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
                    
#                     result[:, :, 3] = mask
#                     result=cv2.addWeighted(color_image_croped,0.5,image,0.7,0)
#                     images=np.hstack((color_image_croped,result,depth_colormap_croped))
#                 except:
#                     images = np.hstack((color_image,depth_colormap))
                #depth = depth_frame.get_distance(int(x+(w-x)/2), int(y+(h-y)/2))
                #dx ,dy, dz = rs.rs2_deproject_pixel_to_point(color_intrin, [x,y], depth)
                #distance = math.sqrt(((dx)**2) + ((dy)**2) + ((dz)**2))
                distance = 0.0
                cv2.rectangle(color_image,(int(x),int(y)),(int(w),int(h)),(0,255,0),2)
                cv2.rectangle(color_image,(int(x),int(y+10)),(int(w),int(y)-15),(0,255,0),-1)
                cv2.rectangle(color_image,(int(x),int(y)),(int(w),int(y)-15),(0,255,0),2)
                cv2.putText(color_image," "+name+" : "+str(round(confidence,1))+", dis: "+ str(round(distance,2)),(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
                #time.sleep(1)
            # except:
            #     continue
            images = np.hstack((color_image,result,depth_colormap))
            
            #croped_images=np.hstack((result))
        else:
            images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        #cv2.imshow('RealSense', images)
        cv2.imshow('RealSense croped', images)
        cv2.waitKey(1)
        #if n>100:
        #    print("ready")
        #    cv2.imwrite('rgb_test_1.jpg', color_image)
        #    cv2.imwrite('depth_test_1.jpg', depth_colormap)
        #    break

            

finally:

    # Stop streaming
    pipeline.stop()

Using cache found in /home/robot/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-4-25 torch 1.11.0+cu102 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11017MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 480 and the array at index 1 has size 476

In [19]:
import pyrealsense2 as rs
import numpy as np
import cv2
import torch
import pandas as pd
import time

def get_filter(depth_colormap,variance):
    #RGB
    Rmax,Gmax,Bmax=255,255,255
    try:
        R=depth_colormap[0].mean()
        G=depth_colormap[1].mean()
        B=depth_colormap[2].mean()
        dmax=math.sqrt((Gmax-G)**2+(Bmax-B)**2)
        dmin=math.sqrt((G)**2+(B)**2)
        theta_1=math.atan(G/B)
        theta_2=math.atan((Gmax-G)/(Bmax-B))
        theta_3=math.asin((Rmax-R)/(dmax))
        theta_4=math.asin(R/dmin)
        #print(theta_1,theta_2,theta_3,theta_4)
        alpha=variance*math.cos(theta_3)
        gamma=variance*math.cos(theta_4)
        Rlow=R-variance*math.cos(theta_3)
        Glow=G-gamma*math.cos(theta_1)
        Blow=B-gamma*math.sin(theta_1)
        Rhigh=R+variance*math.sin(theta_4)
        Ghigh=G+alpha*math.sin(theta_2)
        Bhigh=B+alpha*math.cos(theta_2)
        return (int(Rhigh),int(Ghigh),int(Bhigh)),(int(Rlow),int(Glow),int(Blow))
    except :
        #print("error")
        return (18, 255, 255),(1, 190, 200)

def bincount_app(a):
    a2D = a.reshape(-1,a.shape[-1])
    col_range = (256, 256, 256) # generically : a2D.max(0)+1
    a1D = np.ravel_multi_index(a2D.T, col_range)
    return np.unravel_index(np.bincount(a1D).argmax(), col_range)

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom

x1=100
y1=100
light_blue = (1, 190, 200)
dark_blue = (18, 255, 255)
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)
n=0
try:
    while True:
        n+=1
        #print(n)
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        height, width= depth_image.shape
        #print(height, width)
        depth_image = depth_image[y1:height-y1+10, x1:width-130]
        height, width = depth_image.shape
        #print(height, width)
        #break
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.3), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            depth_colormap = cv2.resize(depth_colormap, dsize=(color_colormap_dim[1], color_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            hsv_real = cv2.cvtColor(depth_colormap, cv2.COLOR_RGB2HSV)
            color_max,color_min=get_filter(depth_colormap,150)
            #color_max=(0,0,255)
            #color_min=(0,0,0)
            #color_min = (1, 190, 200)
            #color_max = (18, 255, 255)
            mask = cv2.inRange(hsv_real, color_min, color_max)
            image = cv2.bitwise_and(depth_colormap, depth_colormap, mask=mask)
            mask = np.where((image==(0,0,0)).all(axis=2), 0, 255).astype(np.uint8)
            # put mask into alpha cha
            result = image.copy()
            result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
            result[:, :, 3] = mask
            result=cv2.addWeighted(color_image,0.5,image,0.7,0)
            results = model(color_image)
            frame_obj=[]
#             for i in range(int(0),int(300)):
#                 column=[]
#                 for j in range(int(0),int(300)):
#                     column.append(depth_frame.get_distance(i,j))
#                 frame_obj.append(column)
#             distance_frame=np.array(frame_obj)
            contours=results.pandas().xyxy[0][results.pandas().xyxy[0]['confidence']>0.6]
            for x,y,w,h,name,confidence in zip(contours['xmin'].to_numpy().tolist(),contours['ymin'].to_numpy().tolist(),contours['xmax'].to_numpy().tolist(),contours['ymax'].to_numpy().tolist(),contours['name'].to_numpy().tolist(),contours['confidence'].to_numpy().tolist()):
                #depth = depth_frame.get_distance(int(x+(w-x)/2), int(y+(h-y)/2))
                #dx ,dy, dz = rs.rs2_deproject_pixel_to_point(color_intrin, [x,y], depth)
                #distance = math.sqrt(((dx)**2) + ((dy)**2) + ((dz)**2))
                frame_obj=[]
                for i in range(int(x),int(w)):
                    column=[]
                    for j in range(int(y),int(h)):
                        column.append(depth_frame.get_distance(i,j))
                    frame_obj.append(column)
                dis_frame=np.array(frame_obj)
                distance=dis_frame[dis_frame<dis_frame.mean()*2.0].mean()
                cv2.rectangle(color_image,(int(x),int(y)),(int(w),int(h)),(0,255,0),2)
                cv2.rectangle(color_image,(int(x),int(y+10)),(int(w),int(y)-15),(0,255,0),-1)
                cv2.rectangle(color_image,(int(x),int(y)),(int(w),int(y)-15),(0,255,0),2)
                l=int(x+((w-x)/2))
                m=int(y+((h-y)/2))
                print(x,y,w,h,l,m)
                cv2.line(color_image, (l,m), (l+20,m), (0,0,255), 2)
                cv2.line(color_image, (l,m-20), (l,m), (0,255,0), 2)
                cv2.line(color_image, (l,m), (l-5,m+5), (255,0,0), 2)
                cv2.putText(color_image," "+name+" : "+str(round(confidence,1))+", dis: "+ str(round(distance,2)),(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
                #x line

                
            images = np.hstack((color_image,depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
        #if n>100:
        #    print("ready")
        #    cv2.imwrite('rgb_test_1.jpg', color_image)
        #    cv2.imwrite('depth_test_1.jpg', depth_colormap)
        #    break
finally:

    # Stop streaming
    pipeline.stop()

Using cache found in /home/robot/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-4-25 torch 1.11.0+cu102 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11017MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


38.753204345703125 0.0 640.0 476.6268310546875 339 238
558.55126953125 253.7481689453125 639.27880859375 360.7491455078125 598 307
83.54498291015625 0.0 635.2393798828125 477.596923828125 359 238
85.7183837890625 0.0 631.9176635742188 477.2377014160156 358 238
558.869140625 254.65745544433594 639.71533203125 365.16937255859375 599 309
560.0897827148438 254.35076904296875 639.7534790039062 364.68182373046875 599 309
33.826873779296875 0.0 638.49609375 476.605712890625 336 238
25.59716796875 0.0 635.5789184570312 477.2120361328125 330 238
560.017333984375 254.69334411621094 639.8067626953125 364.236083984375 599 309
21.921356201171875 0.0 635.375732421875 477.39678955078125 328 238
559.9905395507812 254.71534729003906 639.8881225585938 364.5101623535156 599 309
21.31414794921875 0.0 635.6111450195312 476.0699462890625 328 238
559.2650146484375 254.47496032714844 639.74267578125 364.6966247558594 599 309
23.91436767578125 0.0 633.1788940429688 477.1773376464844 328 238
558.87353515625 254

181.5578155517578 0.09722900390625 550.90283203125 477.6059875488281 366 238
179.41259765625 0.26953125 541.0441284179688 476.4167175292969 360 238
178.21197509765625 0.0 545.5936279296875 477.8655090332031 361 238
178.6396026611328 0.0 550.7969970703125 477.5576477050781 364 238
175.4591064453125 0.0 546.2406616210938 476.90484619140625 360 238
233.36578369140625 201.74676513671875 355.34283447265625 437.6956787109375 294 319
240.4745330810547 208.1271209716797 354.444091796875 421.72784423828125 297 314
245.17164611816406 213.00193786621094 360.7816467285156 447.580322265625 302 330
246.11837768554688 215.22451782226562 358.7166442871094 442.0827941894531 302 328
249.3682098388672 220.9786376953125 362.92840576171875 455.88739013671875 306 338
250.92355346679688 224.32061767578125 362.4165954589844 461.85736083984375 306 343
250.22265625 225.5438232421875 362.808837890625 460.8797607421875 306 343
151.27664184570312 0.127960205078125 538.5880126953125 477.6050109863281 344 238
247.80

221.54510498046875 172.5782470703125 366.10931396484375 407.440673828125 293 290
548.7273559570312 254.9205322265625 639.9094848632812 365.3060302734375 594 310
220.8335723876953 171.45193481445312 366.42572021484375 408.4299011230469 293 289
548.3748168945312 255.46795654296875 640.0 369.37017822265625 594 312
223.76101684570312 172.4150848388672 368.9921569824219 405.354736328125 296 288
225.47125244140625 173.5850830078125 370.4776611328125 411.5496826171875 297 292
548.0587158203125 254.79701232910156 640.0 382.6338195800781 594 318
230.97671508789062 173.2401123046875 376.5957946777344 414.64892578125 303 293
237.15145874023438 170.89862060546875 385.5174255371094 412.94757080078125 311 291
548.2035522460938 254.26382446289062 640.0 393.8794860839844 594 324
548.1619873046875 253.74554443359375 640.0 405.57159423828125 594 329
243.37863159179688 169.88320922851562 386.6047668457031 408.9784851074219 314 289
549.0979614257812 254.45486450195312 640.0 419.8039855957031 594 337
549.8

222.94126892089844 180.45205688476562 340.07952880859375 397.2321472167969 281 288
322.3370361328125 40.04850769042969 639.6856689453125 479.70361328125 481 259
214.03411865234375 179.27557373046875 339.98028564453125 398.77728271484375 277 289
319.8451232910156 39.797698974609375 640.0 478.0317687988281 479 258
215.50111389160156 177.47018432617188 338.7012939453125 399.5269470214844 277 288
318.7159423828125 39.753387451171875 639.8131713867188 478.5920715332031 479 259
218.2076873779297 178.53506469726562 337.26513671875 400.7289733886719 277 289
321.86798095703125 40.24601745605469 640.0 480.0 480 260
220.21051025390625 182.4483642578125 339.005859375 404.98516845703125 279 293
321.3431701660156 40.513031005859375 640.0 480.0 480 260
329.8675537109375 39.378265380859375 640.0 477.3219909667969 484 258
232.99014282226562 185.42088317871094 341.6561584472656 406.4476318359375 287 295
345.76446533203125 40.415435791015625 639.4618530273438 476.0253601074219 492 258
244.26654052734375 

...

227.12242126464844 9.128448486328125 638.6456909179688 469.77764892578125 432 239
235.83013916015625 8.66357421875 639.8466186523438 471.0676574707031 437 239
241.84072875976562 5.355010986328125 639.2930908203125 471.3698425292969 440 238
248.91140747070312 4.716796875 639.29248046875 473.18841552734375 444 238
247.4886932373047 4.780975341796875 638.8096313476562 472.4270324707031 443 238
247.2271728515625 5.0222320556640625 638.0562744140625 472.703857421875 442 238
248.1793975830078 6.204803466796875 638.7438354492188 471.5952453613281 443 238
241.95883178710938 9.057235717773438 640.0 472.0023193359375 440 240
188.5634765625 273.1230163574219 228.35543823242188 327.6178894042969 208 300
232.21450805664062 13.09869384765625 640.0 475.487060546875 436 244
223.81459045410156 2.41156005859375 638.4179077148438 475.1287841796875 431 238
224.44688415527344 6.94720458984375 635.9578247070312 473.7580871582031 430 240
224.2791748046875 5.9754180908203125 637.7750854492188 475.897827148437

216.32122802734375 12.004837036132812 636.0213623046875 476.60430908203125 426 244
216.14418029785156 10.535049438476562 639.8574829101562 476.52435302734375 428 243
213.06878662109375 8.048294067382812 635.2080688476562 473.00347900390625 424 240
208.5021209716797 7.45635986328125 639.240478515625 475.5657043457031 423 241
209.07431030273438 8.741592407226562 640.0 472.060791015625 424 240
209.72091674804688 8.1651611328125 637.66357421875 476.31610107421875 423 242
209.3241424560547 7.8821868896484375 637.4182739257812 475.8519287109375 423 241
205.43222045898438 8.910842895507812 640.0 474.33636474609375 422 241
207.45726013183594 8.057174682617188 638.8007202148438 475.00823974609375 423 241
214.87461853027344 12.642227172851562 638.0402221679688 469.97015380859375 426 241
212.21185302734375 8.352157592773438 633.9549560546875 474.41204833984375 423 241
211.60458374023438 15.039474487304688 638.109130859375 475.9697265625 424 245
209.82606506347656 13.415679931640625 638.9137573242

212.06399536132812 53.48553466796875 557.6092529296875 479.974365234375 384 266
549.0926513671875 255.59552001953125 639.596923828125 427.80010986328125 594 341
213.18666076660156 51.846343994140625 562.2069091796875 479.4770202636719 387 265
549.3984375 255.44021606445312 639.60107421875 420.9009094238281 594 338
188.2715606689453 272.2886047363281 221.70921325683594 329.6849670410156 204 300
214.5607147216797 51.952239990234375 558.6851196289062 480.0 386 265
549.5931396484375 255.21823120117188 639.609130859375 426.5184020996094 594 340
188.20443725585938 272.6175842285156 222.98529052734375 328.9976501464844 205 300
549.5608520507812 255.265869140625 639.6569213867188 424.36785888671875 594 339
215.29652404785156 47.789276123046875 556.6060791015625 480.0 385 263
188.82504272460938 272.743896484375 224.444580078125 329.04864501953125 206 300
217.81759643554688 43.32337951660156 554.88232421875 479.40740966796875 386 261
549.352294921875 255.00820922851562 639.58447265625 424.532073

35.229522705078125 0.0 639.70556640625 478.95147705078125 337 239
36.097076416015625 0.0 640.0 478.04595947265625 338 239
33.97003173828125 0.0 640.0 478.02764892578125 336 239
31.777557373046875 0.0 638.5789794921875 477.8465576171875 335 238
37.117462158203125 0.0 640.0 478.176025390625 338 239
35.76898193359375 0.0 640.0 478.3634033203125 337 239
37.219757080078125 0.0 640.0 478.84747314453125 338 239
36.722259521484375 0.0 639.88916015625 479.31903076171875 338 239
37.86126708984375 0.0 640.0 479.5556640625 338 239
34.654510498046875 0.0 640.0 479.66461181640625 337 239
41.85601806640625 0.0 640.0 479.86114501953125 340 239
36.156646728515625 0.0 640.0 480.0 338 240
37.387908935546875 0.0 640.0 480.0 338 240
35.510589599609375 0.0 640.0 479.6438903808594 337 239
37.177032470703125 0.0 638.4217529296875 480.0 337 240
34.206024169921875 0.0 640.0 480.0 337 240
41.8355712890625 0.0 640.0 480.0 340 240
37.787109375 0.0 640.0 479.57171630859375 338 239
37.174468994140625 0.0 639.0142822

KeyboardInterrupt: 